В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [6]:
# расчеты строго по формуле доли двух независимых выборок см.конспект

import scipy
#import numpy as np
from scipy.stats import norm

n1 = 34
n2 = 16

a = 10
b = 4

p1 = float(a) / n1
p2 = float(b) / n2 
P = float(p1*n1 + p2*n2) / (n1 + n2)

Z = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

p = 2 * (1 - scipy.stats.norm.cdf(np.abs(Z)))
p


0.74586091745047067

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

    логистическая регрессия по признакам X1,X2,X3
    логистическая регрессия по признакам X4,X5,X6

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10^-8, нужно ввести 8).

In [8]:
import pandas as pd
from sklearn.cross_validation import train_test_split

/home/sigor/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
bn = pd.read_csv('banknotes.txt', sep='\t')
bn.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [26]:
bn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
X1      200 non-null float64
X2      200 non-null float64
X3      200 non-null float64
X4      200 non-null float64
X5      200 non-null float64
X6      200 non-null float64
real    200 non-null int64
dtypes: float64(6), int64(1)
memory usage: 11.0 KB


In [24]:
bn_train, bn_test = train_test_split(bn, test_size=50, random_state=1)

In [29]:
bn_train_x = bn_train.drop('real', axis=1)
bn_train_y = bn_train['real']

bn_test_x = bn_test.drop('real', axis=1)
bn_test_y = bn_test['real']

In [34]:
from sklearn.linear_model import LogisticRegression
classifier1 = LogisticRegression()
classifier2 = LogisticRegression()

In [35]:
data1 = ['X1', 'X2', 'X3']
data2 = ['X4', 'X5', 'X6']

classifier1.fit(bn_train_x[data1], bn_train_y)
classifier2.fit(bn_train_x[data2], bn_train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [38]:
pred1 = classifier1.predict(bn_test_x[data1])
pred2 = classifier2.predict(bn_test_x[data2])

In [43]:
print pred1
print bn_test_y.real

[1 1 1 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0
 0 0 0 1 0 0 1 1 0 1 0 0 0]
[1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 0
 1 1 0 0 0 0 1 1 1 0 1 0 0]


In [45]:
print pred2
print bn_test_y.real

[1 1 1 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 0
 1 1 0 0 0 0 1 1 1 0 1 0 0]
[1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 0
 1 1 0 0 0 0 1 1 1 0 1 0 0]
